## Connect to remote postgresql on usyd

In [1]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras

YOUR_PW     = '490419348'
DB_LOGIN    = 'y20s1d2x01_jhwa3223'

sql_url = 'postgres+psycopg2://{}:{}@soitpw11d59.shared.sydney.edu.au/{}'
engine = create_engine(sql_url.format(DB_LOGIN, YOUR_PW, DB_LOGIN), echo = False)

conn = engine.connect()

## Pull data from web API

In [3]:
import pandas as pds
import urllib

engine.execute("""
    SET search_path TO assignment;
""")

engine.execute("""
    DROP TABLE IF EXISTS assignment.covid19_cases;
    
    
    CREATE TABLE assignment.covid19_cases
    (
        Id BIGSERIAL PRIMARY KEY,
        notification_date DATE,
        postcode BIGINT,
        lhd_2010_code CHAR(4),
        lhd_2010_name TEXT,
        lga_code19 DOUBLE PRECISION,
        lga_name19 TEXT
    );
""")


url = 'https://data.nsw.gov.au/data/api/3/action/datastore_search?offset={}&resource_id=21304414-1ff1-4243-a5d2-f52778048b29'

offset = 0
total = 1

# Assumption: The data can only grow
while offset < total:
    raw_df = pds.read_json(urllib.request.urlopen(url.format(offset)))
    
    df = pds.DataFrame(raw_df.loc["records", "result"])
    print(df)
    df.to_sql('covid19_cases', con = conn, if_exists = 'append', index = False)
    
    offset += 100
    total = raw_df.loc["total"]["result"]

   notification_date  postcode lhd_2010_code          lhd_2010_name  \
0         2020-01-22      2134          X700                 Sydney   
1         2020-01-24      2121          X760        Northern Sydney   
2         2020-01-25      2071          X760        Northern Sydney   
3         2020-01-25      2033          X720   South Eastern Sydney   
4         2020-01-30      2350          X800     Hunter New England   
..               ...       ...           ...                    ...   
95        2020-03-11      2766          X740         Western Sydney   
96        2020-03-11      2020          X720   South Eastern Sydney   
97        2020-03-11      2114          X760        Northern Sydney   
98        2020-03-12      2756          X750  Nepean Blue Mountains   
99        2020-03-12      2120          X760        Northern Sydney   

    lga_code19             lga_name19  
0      11300.0            Burwood (A)  
1      16260.0         Parramatta (C)  
2      14500.0        Ku-ri

In [4]:
conn.execute("""
    SELECT *
      FROM covid19_cases
     LIMIT 25;
""").fetchall()

[(1, datetime.date(2020, 1, 22), 2134, 'X700', 'Sydney', 11300.0, 'Burwood (A)'),
 (2, datetime.date(2020, 1, 24), 2121, 'X760', 'Northern Sydney', 16260.0, 'Parramatta (C)'),
 (3, datetime.date(2020, 1, 25), 2071, 'X760', 'Northern Sydney', 14500.0, 'Ku-ring-gai (A)'),
 (4, datetime.date(2020, 1, 25), 2033, 'X720', 'South Eastern Sydney', 16550.0, 'Randwick (C)'),
 (5, datetime.date(2020, 1, 30), 2350, 'X800', 'Hunter New England', 10130.0, 'Armidale Regional (A)'),
 (6, datetime.date(2020, 2, 21), 2010, 'X720', 'South Eastern Sydney', 17200.0, 'Sydney (C)'),
 (7, datetime.date(2020, 2, 26), 2073, 'X760', 'Northern Sydney', 14500.0, 'Ku-ring-gai (A)'),
 (8, datetime.date(2020, 2, 27), 2070, 'X760', 'Northern Sydney', 14500.0, 'Ku-ring-gai (A)'),
 (9, datetime.date(2020, 2, 28), 2750, 'X750', 'Nepean Blue Mountains', 16350.0, 'Penrith (C)'),
 (10, datetime.date(2020, 2, 29), 2077, 'X760', 'Northern Sydney', 14000.0, 'Hornsby (A)'),
 (11, datetime.date(2020, 3, 1), 2163, 'X710', 'South 

In [4]:
conn.execute("""
    SELECT COUNT(*)
      FROM covid19_cases
     LIMIT 25;
""").fetchall()

[(3059,)]

## Clean the data

Is there any null data?

In [5]:
conn.execute("""
    SELECT COUNT(*)
      FROM covid19_cases
     WHERE NOT (covid19_cases IS NOT NULL);
""").fetchall()

[(136,)]

Have a glimpse at them

In [6]:
conn.execute("""
    SELECT *
      FROM covid19_cases
     WHERE NOT (covid19_cases IS NOT NULL)
     LIMIT 10;
""").fetchall()

[(48, datetime.date(2020, 3, 7), 2091, '    ', '', None, ''),
 (2823, datetime.date(2020, 4, 9), 3004, '    ', '', None, ''),
 (161, datetime.date(2020, 3, 14), 9990, '    ', '', None, ''),
 (194, datetime.date(2020, 3, 15), 4566, '    ', '', None, ''),
 (462, datetime.date(2020, 3, 20), 5158, '    ', '', None, ''),
 (487, datetime.date(2020, 3, 20), None, '    ', '', None, ''),
 (524, datetime.date(2020, 3, 20), 1871, '    ', '', None, ''),
 (539, datetime.date(2020, 3, 20), 2612, '    ', '', None, ''),
 (612, datetime.date(2020, 3, 21), None, '    ', '', None, ''),
 (679, datetime.date(2020, 3, 21), 0, '    ', '', None, '')]

Remove them from database

In [7]:
conn.execute("""
    DELETE FROM covid19_cases
     WHERE NOT (covid19_cases IS NOT NULL);
""")

In [8]:
conn.execute("""
    SELECT COUNT(*)
      FROM covid19_cases
""").fetchall()

[(2923,)]

In [9]:
conn.execute("""
    SELECT *
      FROM covid19_cases
     LIMIT 10;
""").fetchall()

[(1, datetime.date(2020, 1, 22), 2134, 'X700', 'Sydney', 11300.0, 'Burwood (A)'),
 (2, datetime.date(2020, 1, 24), 2121, 'X760', 'Northern Sydney', 16260.0, 'Parramatta (C)'),
 (3, datetime.date(2020, 1, 25), 2071, 'X760', 'Northern Sydney', 14500.0, 'Ku-ring-gai (A)'),
 (4, datetime.date(2020, 1, 25), 2033, 'X720', 'South Eastern Sydney', 16550.0, 'Randwick (C)'),
 (5, datetime.date(2020, 1, 30), 2350, 'X800', 'Hunter New England', 10130.0, 'Armidale Regional (A)'),
 (6, datetime.date(2020, 2, 21), 2010, 'X720', 'South Eastern Sydney', 17200.0, 'Sydney (C)'),
 (7, datetime.date(2020, 2, 26), 2073, 'X760', 'Northern Sydney', 14500.0, 'Ku-ring-gai (A)'),
 (8, datetime.date(2020, 2, 27), 2070, 'X760', 'Northern Sydney', 14500.0, 'Ku-ring-gai (A)'),
 (9, datetime.date(2020, 2, 28), 2750, 'X750', 'Nepean Blue Mountains', 16350.0, 'Penrith (C)'),
 (10, datetime.date(2020, 2, 29), 2077, 'X760', 'Northern Sydney', 14000.0, 'Hornsby (A)')]

## Create indexes on data

In [10]:
engine.execute("""
    CREATE INDEX postcode_index ON covid19_cases (postcode);
""")